In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pair Programming: MongoDB exercises to build familiarity

### Objectives
- Start and stop clients
- Create databases and collections
- Add, update, and drop objects
- Load and manage twitter tweets in Mongo

Start up mongo session, and import MongoClient from pymongo.  Pip install if necessary.  Use MongoClient() to create a new client object.  

In [ ]:
#!brew services start mongodb
#!brew services stop mongodb

In [8]:
from pymongo import MongoClient

In [9]:
client = MongoClient()

Use client.TAB to learn about the properties you can call on client.  Can you get the host, port, address, database names of the databases currently stored there?

In [10]:
client.HOST

'localhost'

In [11]:
client.PORT

27017

In [12]:
client.address

('localhost', 27017)

In [13]:
client.database_names()

['admin', 'local']

Pick one of the databases and create an object representing that database.

In [14]:
db = client.my_cool_database

Show the collections that are in there.

In [15]:
db.collection_names()

[]

In [16]:
db.drop_collection("funky_test")

{'code': 26,
 'codeName': 'NamespaceNotFound',
 'errmsg': 'ns not found',
 'ok': 0.0}

Create a new collection with a name of your choice.

In [17]:
db.create_collection("funky_test")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'my_cool_database'), 'funky_test')

In [18]:
db.collection_names()

['funky_test']

Get one of the collections and create a new object representing it.

In [19]:
funky_test = db.get_collection('funky_test')

In [20]:
type(funky_test)

pymongo.collection.Collection

How many documents are in there?

In [21]:
funky_test.count()

0

Insert a document of your creation into your collection.

In [22]:
funky_test.insert_one({"username": "erb", "name": "erbsman", "email": "erbstermeistro@gmail.com"})

Use InsertOne, DeleteOne, and ReplaceOne to insert, delete, and replace three documents of your own creation.  Put the jobs in a list, and submit it in bulk.

In [23]:
funky_test.bulk_write?

In [24]:
from pymongo import InsertOne, DeleteOne, ReplaceOne

In [25]:
requests = [InsertOne({'y': 1}), DeleteOne({'x': 1}), ReplaceOne({'w': 1}, {'z': 1}, upsert=True)]

In [26]:
result = db.funky_test.bulk_write(requests)

In [27]:
funky_test.insert_one({'y':1})

In [29]:
for i in funky_test.find():
    print(i)

{'username': 'erb', 'email': 'erbstermeistro@gmail.com', '_id': ObjectId('59930a0f337c320ee8303e96'), 'name': 'erbsman'}
{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e97')}
{'z': 1, '_id': ObjectId('59930a0f2367c43ecfdecd1f')}
{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e98')}


How many documents are in the collection now?

In [30]:
funky_test.count()

4

Find one of them.

In [31]:
funky_test.find_one({"y":1})
# user_erb = users.find_one({"username": "erb"})

{'_id': ObjectId('59930a0f337c320ee8303e97'), 'y': 1}

In [34]:
for i in funky_test.find({'y':1}):
    print(i)

{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e97')}
{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e98')}


Print out all documents in your collection.

In [33]:
for funky_doc in funky_test.find():
    print(funky_doc)

{'username': 'erb', 'email': 'erbstermeistro@gmail.com', '_id': ObjectId('59930a0f337c320ee8303e96'), 'name': 'erbsman'}
{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e97')}
{'z': 1, '_id': ObjectId('59930a0f2367c43ecfdecd1f')}
{'y': 1, '_id': ObjectId('59930a0f337c320ee8303e98')}


Append the ID's of each document to an empty list.  Then show the list.

In [ ]:

doc_ids = []
for funky_doc in funky_test.find():
    doc_ids.append(funky_doc['_id'])
  
doc_ids

In [ ]:
funky_test.find({'y':1})

Finally, drop your new collection.

In [ ]:
db.drop_collection("funky_test")

Get some Twitter.  For access tokens, go to [Twitter's API site](https://apps.twitter.com) and load your consumer_key, consumer_secret, access_token, and access_token_secret into .twitter_config on your home directory.  See yesterday's Twittter notebook.

In [ ]:
# pip install requests_oauthlib
import requests
from requests_oauthlib import OAuth1
import cnfg
from pprint import pprint
import json
import tweepy

In [ ]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [ ]:

# ?? this is an instance 

response = requests.get("https://api.twitter.com/1.1/statuses/user_timeline.json",
                        auth=oauth)
tweets = response.json()


In [ ]:
db = client.funky_test

In [ ]:
# Our collection: 
tweets = db.new_tweet

In [ ]:
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [ ]:
tweets

In [ ]:
max_tweets=2

#Tweepy Cursor handles pagination .. 

for tweet in tweepy.Cursor(api.search,q="Cubs").items(max_tweets):
    pprint(tweet)

In [ ]:
results=[]


for tweet in tweepy.Cursor(api.search,q="Cubs").items(10):
    results.append(tweet)

In [ ]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert_one(data)

In [ ]:
tweets.find_one()

In [ ]:
for tweet in tweets.find():
    print tweet['tweet']

In [ ]:
for tweet in tweets.find():
    print tweet['datetime']